In [1]:
import torch
import random

import pandas as pd
import numpy as np

from etna.datasets.tsdataset import TSDataset
from etna.pipeline import AutoRegressivePipeline

import warnings


def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


warnings.filterwarnings("ignore")

/Users/buchkovv/mlfcs/.venv_new/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/buchkovv/mlfcs/.venv_new/lib/python3.10/site-packages/etna/settings.py:51: UserWarning: etna[chronos] is not available, to install it, run `pip install etna[chronos]`
  warnings.warn("etna[chronos] is not available, to install it, run `pip install etna[chronos]`")
/Users/buchkovv/mlfcs/.venv_new/lib/python3.10/site-packages/etna/settings.py:66: UserWarning: etna[timesfm] is not available, to install it, run `pip install etna[timesfm]`
  warnings.warn("etna[timesfm] is not available, to install it, run `pip install etna[timesfm]`")
/Users/buchkovv/mlfcs/.venv_new/lib/python3.10/site-packages/etna/settings.py:74: UserWarning: wandb is not available, to install it, run `pip install etna[wandb]`
  warnings.warn("wandb is 

In [2]:
from config.experiment_config import ExperimentConfig
from config.model_config import ModelConfig

model_config = ModelConfig()
experiment_config = ExperimentConfig()

In [3]:
data = pd.read_csv(experiment_config.PATH_DATA / experiment_config.DATASET.value)
data["datetime"] = pd.to_datetime(data["datetime"])
data = data.sort_values(by="datetime")
data = data.set_index("datetime")
data.index = data.index.tz_localize(None)

data["segment"] = "segment_A"

In [4]:
returns = data.loc[:, experiment_config.RETURN_COLUMN].iloc[1:]

vols = data.loc[:, experiment_config.VOL_COLUMN].iloc[1:]

feature_columns = data.columns.difference(
    [experiment_config.RETURN_COLUMN, experiment_config.VOL_COLUMN]
).tolist()
features = data[feature_columns].shift(1).iloc[1:]

if model_config.n_features is None:
    model_config.n_features = len(feature_columns)

if model_config.n_unique_features is None:
    unique_columns = ["_".join(column.split("_")[:-1]) for column in feature_columns]
    model_config.n_unique_features = np.unique(unique_columns).shape[0]

data = data.reset_index()
data = data.rename(columns={"datetime": "timestamp"})
data = data.set_index("timestamp")

In [5]:
from etna.models.nn import NBeatsGenericModel


def train_n_beats(train_df: pd.DataFrame, test_df: pd.DataFrame):
    set_seed()

    df = TSDataset.to_dataset(train_df)
    ts = TSDataset(df, freq="h")

    model_deepar = NBeatsGenericModel(
        input_size=train_df.shape[0],
        output_size=1,
        loss="mse",
        stacks=30,
        layers=4,
        layer_size=256,
        trainer_params=dict(max_epochs=500),
    )

    pipeline_deepar = AutoRegressivePipeline(
        model=model_deepar,
        horizon=test_df.shape[0],
    )

    _ = pipeline_deepar.fit(ts=ts)
    forecast = pipeline_deepar.forecast()
    pred_vol = forecast.loc[:, ("segment_A", "target")].to_numpy()

    return pred_vol

In [6]:
def run_backtest():
    step = experiment_config.ROLLING_STEP_DAYS

    train_start = experiment_config.TRAIN_START_DATE
    train_end = experiment_config.FIRST_TRAIN_END_DATE

    rolling_results = []
    while train_end <= data.index[-1]:
        last_train_end = train_end

        train_data = (
            data.resample("h").ffill().loc[train_start:train_end, :].reset_index()
        )
        train_data = train_data.rename(columns={"vol": "target"})

        train_start = (
            train_start + pd.Timedelta(days=step)
            if experiment_config.EXPANDING
            else train_start
        )
        train_end = train_end + pd.Timedelta(days=step)

        test_data = data.loc[
            last_train_end + pd.Timedelta(milliseconds=1) : train_end, :
        ].reset_index()

        if len(train_data) > 0 and len(test_data) > 0:
            forecast = train_n_beats(
                train_df=train_data,
                test_df=test_data,
            )

            rolling_results.append(
                [last_train_end, test_data["vol"].to_numpy(), forecast]
            )

    return pd.DataFrame(
        rolling_results, columns=["datetime", "true_vols", "model_preds"]
    ).set_index("datetime")

In [7]:
results = run_backtest()

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params
------------------------------------
0 | model | NBeats    | 506 K 
1 | loss  | NBeatsMSE | 0     
------------------------------------
506 K     Trainable params
0         Non-trainable params
506 K     Total params
2.027     Total estimated model params size (MB)


Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 59.33it/s, loss=3.91e-06]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 38.18it/s, loss=3.91e-06]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params
------------------------------------
0 | model | NBeats    | 518 K 
1 | loss  | NBeatsMSE | 0     
------------------------------------
518 K     Trainable params
0         Non-trainable params
518 K     Total params
2.074     Total estimated model params size (MB)


...

Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 24.77it/s, loss=1.91e-05]


GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name  | Type      | Params
------------------------------------
0 | model | NBeats    | 518 K 
1 | loss  | NBeatsMSE | 0     
------------------------------------
518 K     Trainable params
0         Non-trainable params
518 K     Total params
2.074     Total estimated model params size (MB)


Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 48.23it/s, loss=2.86e-05]

`Trainer.fit` stopped: `max_epochs=500` reached.


Epoch 499: 100%|██████████| 1/1 [00:00<00:00, 29.83it/s, loss=2.86e-05]


In [8]:
results.to_csv("NBeats_Features.csv")